## Problem 1
Use the rate given by

$$r_{\text{est}}(t)=r_0+\int_{0}^{\infty}d\tau D(\tau)s(t-\tau)$$

with $r_0$ = 50 Hz and

$$D(\tau) = -\cos\left(\frac{2\pi (\tau - 20 \text{ms})}{140 \text{ms}}\right)\exp\left(-\frac{\tau}{60 \text{ms}}\right) \text{Hz/ms}$$

to predict the response of a neuron of the electrosensory lateral-line lobe to a stimulus. The above equation is an approximation for the linear kernel obtained from the spike-triggered average shown in figure 1.9. Use an approximate Gaussian white noise stimulus constructed by choosing a stimulus value every 10 ms ($\Delta t$ = 10 ms) from a Gaussian distribution with zero mean and variance $\sigma_s^2/\Delta t$,
with $\sigma_s^2$ = 10. Compute the firing rate over a 10 s period. From the
results, compute the firing rate-stimulus correlation function $Q_{rs}(\tau)$.
Using

$$D(\tau)=\frac{Q_{rs}(\tau)}{\sigma_s^2}=\frac{\langle r \rangle C(\tau)}{\sigma_s^2},$$

compare $Q_{rs}(-\tau)/\sigma_s^2$ with the kernel $D(\tau)$ given above.

In [1]:
% TODO

## Problem 2

`c1p8.mat` contains the data described in exercise 8 of chapter 1. Use the spike-triggered average (calculated in that exercise) to construct a linear kernel and use it in

$$r_{\text{est}}(t)=r_0+\int_{0}^{\infty}d\tau D(\tau)s(t-\tau)$$

to provide a model of the response of the H1 neuron. Choose $r_0$ so that the average firing rate predicted by the model in response to the stimulus used for the data matches the actual average firing rate. Use a Poisson generator with the computed rate to generate a synthetic spike train from this linear estimate of the firing rate in response to the stimulus `stim`. Plot examples of the actual and synthetic spike trains. How are they similar and how do they differ? Plot the autocorrelation function of the actual and the synthetic spike trains over the range 0 to 100 ms. Why is there a dip at a lag of 2 ms in the autocorrelation of the actual spike train? Is there a dip for the synthetic train too? Plot the interspike interval histogram for both spike trains. Why is there a dip below 6 ms in the histogram for the actual spike train? What are the coefficients of variation for the two spike trains and why might they differ?

In [2]:
% TODO

## Problem 3

`c2p3.mat` contains the responses of a cat LGN cell to two-dimensional visual images (these data are described in Kara, P, Reinagel, P, & Reid, RC (2000) Low response variability in simultaneously recorded retinal, thalamic, and cortical neurons. _Neuron_ **30**:803-817 and were kindly provided by Clay Reid). In the file, `counts` is a vector containing the number of spikes in each 15.6 ms bin, and `stim` contains the 32767, 16 × 16 images that were presented at the corresponding times. Specifically, `stim(x, y, t)` is the stimulus presented at the coordinate `(x,y)` at time-step `t`. Note that `stim` is an `int8` array that must to be converted into `double` using the command `stim=double(stim)` in order to be manipulated within Matlab. Calculate the spike-triggered average images for each of the 12 time steps before each spike and show them all (using the `imagesc` command). Note that in this example, the time bins can contain more than one spike, so the spike-triggered average must be computed by weighting each stimulus by the number of spikes in the corresponding time bin, rather than weighting it by either 1 or 0 depending on whether a spike is present or not. In the averaged images, you should see a central receptive field that reverses sign over time. By summing up the images across one spatial dimension, produce a figure like that of figure 2.25C.

In [3]:
% TODO

## Problem 4
For a Gaussian random variable $x$ with zero mean and standard deviation $\sigma$, prove that

$$\langle xF(\alpha x)\rangle=\alpha\sigma^2\langle F'(\alpha x)\rangle,$$

where $\alpha$ is a constant, $F$ is any function, $F'$ is its derivative,

$$\langle xF(\alpha x)\rangle = \int dx \frac{1}{\sqrt{2\pi}\sigma}\exp\left(-\frac{x^2}{2\sigma^2}\right)xF(\alpha x),$$

and similarly for $\langle F'(\alpha x)\rangle$. This is the basis of the identity 2.64, which can be derived by extending this basic result first to multivariate functions and then to functionals.

In [4]:
% TODO

## Problem 5
Using

$$\epsilon=\epsilon_0(\exp(X/\lambda)-1) \text{  and  } a=-\frac{180^\circ(\epsilon_0 + \epsilon)Y}{\lambda \epsilon \pi},$$

map from cortical coordinates back to visual coordinates and determine what various patterns of activity across the primary visual cortex would “look like”. Ermentrout and Cowan (Ermentrout, GB, & Cowan, J (1979) A mathematical theory of visual hallucination patterns. _Biological Cybernetics_ **34**:137–150) used these results as a basis of a mathematical theory of visual hallucinations. The figure generated by the Matlab program `c2p5.m` shows an illustrative example. This program simulates a plane sine wave of activity across the primary visual cortex with a specified spatial frequency and direction, and then maps it back into retinal coordinates to see what visual pattern would be perceived due to this activity. Consider various other patterns of activity and show the visual hallucinations they would generate.

In [5]:
% TODO

## Problem 6
Perform the integrals

$$L_s = \int dx dy D_s(x,y)A\cos\left(Kx\cos(\Theta)+ Ky\sin(\Theta) - \Phi\right)$$

and

$$L_t(t)=\int_0^\infty d\tau D_\tau(\tau)\cos(\omega(t-\tau))$$

for the case $\sigma_x=\sigma_y=\sigma$ to obtain the results

$$L_s=\frac{A}{2}\exp\left(-\frac{\sigma^2(k^2+K^2)}{2}\right)\left(\cos(\phi-\Phi)\exp\left(\sigma^2kK\cos(\Theta)\right) + \cos(\phi+\Phi)\exp\left(-\sigma^2kK\cos(\Theta)\right)\right)$$

and

$$L_t(t)=\frac{\alpha^6|\omega| \sqrt{\omega^2+4\alpha^2}}{(\omega^2+\alpha^2)^4}\cos(\omega t - \delta)$$

with

$$\delta = 8 \arctan\left(\frac{\omega}{\alpha}\right)+\arctan\left(\frac{2\alpha}{\omega}\right)-\pi$$

From these results, verify the selectivity curves in figures 2.15 and 2.16. In addition, plot $\delta$ as a function of $\omega$.

In [6]:
% TODO

## Problem 7
Numerically compute the spatial part of the linear response of a simple cell with a separable space-time receptive field to a sinusoidal grating, as given by

$$L_s = \int dx dy D_s(x,y)A\cos\left(Kx\cos(\Theta)+ Ky\sin(\Theta) - \Phi\right)$$

Use a stimulus oriented with $\Theta= 0$. For the spatial receptive field kernel, use

$$D_s(x,y)=\frac{1}{2\pi \sigma_x \sigma_y}\exp\left(-\frac{x^2}{2\sigma_x^2}-\frac{y^2}{2\sigma_y^2}\right)\cos(kx-\phi)$$

with $\sigma_x =\sigma_y = 1^\circ$, $\phi=0$, and $1/k = 0.5^\circ$. Plot $L_s$ as a function of $K$ taking $\Phi=0$ and $A = 50$. This determines the spatial frequency selectivity of the cell. What is its preferred spatial frequency? Plot $L_s$ as a function of $\Phi$ taking $1/K = 0.5^\circ$ and $A = 50$. This determines the spatial phase selectivity of the cell. What is its preferred spatial phase?

In [7]:
% TODO

## Problem 8
Consider a complex cell with the spatial part of its response given by $L_1^2 + L_2^2$, where $L_1$ and $L_2$ are linear responses determined by

$$L_s = \int dx dy D_s(x,y)A\cos\left(Kx\cos(\Theta)+ Ky\sin(\Theta) - \Phi\right)$$

with kernels given by

$$D_s(x,y)=\frac{1}{2\pi \sigma_x \sigma_y}\exp\left(-\frac{x^2}{2\sigma_x^2}-\frac{y^2}{2\sigma_y^2}\right)\cos(kx-\phi)$$

with $\sigma_x = \sigma_y = 1^\circ$, and $1/k = 0.5^\circ$; and with $\phi = 0$ for $L_1$ and $\phi = -\pi/2$ for $L_2$. Use a stimulus oriented with $\theta = 0$. Compute and plot $L_1^2 + L_2^2$ as a function of $K$ taking $\Phi = 0$ and $A = 5$. This determines the spatial frequency selectivity of the cell. Compute and plot $L_1^2 + L_2^2$ as a function of $\Phi$ taking $1/K = 0.5^\circ$ and $A = 5$. This determines the spatial phase selectivity of the cell. Does the spatial phase selectivity match what you expect for a complex cell?

In [8]:
% TODO

## Problem 9
Consider the linear temporal response for a simple or complex cell given by

$$L_t(t)=\int_0^\infty d\tau D_\tau(\tau)\cos(\omega(t-\tau))$$

with a temporal kernel given by

$$D_t(\tau)=\alpha\exp(-\alpha\tau)\left(\frac{(\alpha\tau)^5}{5!}-\frac{(\alpha\tau)^7}{7!}\right)$$

with $1/\alpha = 15$ ms. Compute and plot $L_t(t)$ for $\omega = 6\pi/s$. This determines the temporal response of the simple cell. Do not plot the negative part of $L_t(t)$ because the cell cannot fire at a negative rate. Compute and plot $L_t^2(t)$ for $\omega = 6\pi/s$. This determines the temporal response of a complex cell. What are the differences between the temporal responses of the simple and complex cells?

In [9]:
% TODO

## Problem 10
Compute the response of a model simple cell with a separable space-time receptive field to a moving grating $s(x,y,t) = \cos(Kx - \omega t)$. For $D_s$, use

$$D_s(x,y)=\frac{1}{2\pi \sigma_x \sigma_y}\exp\left(-\frac{x^2}{2\sigma_x^2}-\frac{y^2}{2\sigma_y^2}\right)\cos(kx-\phi)$$

with $\sigma_x = \sigma_y = 1^\circ$, $\phi = 0$, and $1/k = 0.5^\circ$. For $D_t$, use

$$D_t(\tau)=\alpha\exp(-\alpha\tau)\left(\frac{(\alpha\tau)^5}{5!}-\frac{(\alpha\tau)^7}{7!}\right)$$

with $1/\alpha = 15$ ms. Compute the linear estimate of the response given by

$$L(t)=\int_0^\infty d\tau\int dx dy D(x, y, \tau)s(x,y,t-\tau)$$

and assume that the actual response is proportional to a rectified version of this linear response estimate. Plot the response as a function of time for $1/K = 1/k = 0.5^\circ$ and $\omega = 8\pi/s$. Plot the response amplitude as a function of $\omega$ for $1/K = 1/k = 0.5^\circ$ and as a function of $K$ for $\omega = 8\pi/s$.

In [10]:
% TODO

## Problem 11
Compute the response of a model complex cell to the moving grating

$$s(x, y, t) = \cos(Kx - \omega t)$$

The complex cell should be modeled by squaring the unrectified linear response estimate of a simple cells with a spatial receptive field given by

$$D_s(x,y)=\frac{1}{2\pi \sigma_x \sigma_y}\exp\left(-\frac{x^2}{2\sigma_x^2}-\frac{y^2}{2\sigma_y^2}\right)\cos(kx-\phi)$$

with $\sigma_x = \sigma_y = 1^\circ$, $\phi = 0$, and $1/k = 0.5^\circ$, and adding this to the square of the unrectified linear response of a second simple cell with identical properties except that its spatial phase preference is $\phi = -\pi/2$ instead of $\phi = 0$. Both linear responses are computed from

$$L(t)=\int_0^\infty d\tau\int dx dy D(x, y, \tau)s(x,y,t-\tau)$$

For both of these, use

$$D_t(\tau)=\alpha\exp(-\alpha\tau)\left(\frac{(\alpha\tau)^5}{5!}-\frac{(\alpha\tau)^7}{7!}\right)$$

with $1/\alpha = 15$ ms for the temporal receptive field. Plot the complex cell response as a function of time for $1/K = 1/k = 0.5^\circ$ and $\omega = 8\pi/s$. Plot the response amplitude as a function of $\omega$ for $1/K = 1/k = 0.5^\circ$ and as a function of $K$ for $\omega = 8\pi/s$.

In [11]:
% TODO

## Problem 12
Construct a model simple cell with the nonseparable space-time receptive field described in the caption of figure 2.21B. Compute its response to the moving grating

$$s(x, y, t) = cos(Kx - \omega t)$$

Plot the amplitude of the response as a function of the velocity of the grating, $\omega/K$, using $\omega = 8\pi/s$ and varying $K$ to obtain a range of both positive and negative velocity values (use negative $K$ values for this). Show that the response is directionally selective.

In [12]:
% TODO

## Problem 13
Construct a model complex cell that is disparity tuned but insensitive to the absolute position of a grating. The complex cell is constructed by summing the squares of the unrectified linear responses of two simple cells, but disparity effects are now included. For this exercise, we ignore temporal factors and only consider the spatial dependence of the response. Each simple cell response is composed of two terms that correspond to inputs coming from the left and right eyes. Because of disparity, the spatial phases of the image of a grating in the two eyes, $\Phi_L$ and $\Phi_R$, may be different. We write the spatial part of the linear response estimate for a grating with the preferred spatial frequency $(k = K)$ and orientation $(\Theta = \theta = 0)$ as

$$L_1 = \frac{A}{2}(\cos(\Phi_L ) + \cos(\Phi_R ))$$

assuming that $\phi = 0$ (this equation is a generalization of equation 2.34). Let the complex cell response be proportional to $L_1^2 + L_2^2$, where $L_2$ is similar to $L_1$ but with the cosine functions replaced by sine functions. Show that the response of this neuron is tuned to the disparity, $\Phi_L - \Phi_R$, but is independent of the absolute spatial phase of the grating, $\Phi_L + \Phi_R$. Plot the response tuning curve as a function of disparity. (See DeAngelis, GC, Ohzawa, I, & Freeman, RD (1991) Depth is encoded in the visual cortex by a specialized receptive field structure. _Nature_ **352**:156–159.)

In [13]:
% TODO

## Problem 14
Determine the selectivity of the LGN receptive field of

$$D_s(x,y)=\pm\left(\frac{1}{2\pi \sigma_{cen}^2}\exp\left(-\frac{x^2+y^2}{2\sigma_{cen}^2}\right)-\frac{B}{2\pi \sigma_{sur}^2}\exp\left(-\frac{x^2+y^2}{2\sigma_{sur}^2}\right)\right)$$

to spatial frequency by computing its integrals when multiplied by the stimulus

$$s = \cos(Kx)$$

for a range of $K$ values. Use $\sigma_c = 0.3^\circ$, $\sigma_s = 1.5^\circ$, $B = 5$, $1/\alpha = 16$ ms, and $1/\beta = 64$ ms, and plot the resulting spatial frequency tuning curve.

In [14]:
% TODO

## Problem 15
Construct the Hubel-Wiesel model of a simple-cell spatial receptive field, as depicted in figure 2.27A. Use difference-of-Gaussian functions given by

$$D_s(x,y)=\pm\left(\frac{1}{2\pi \sigma_{cen}^2}\exp\left(-\frac{x^2+y^2}{2\sigma_{cen}^2}\right)-\frac{B}{2\pi \sigma_{sur}^2}\exp\left(-\frac{x^2+y^2}{2\sigma_{sur}^2}\right)\right)$$

to model the LGN receptive fields. Plot the spatial receptive field of the simple cell constructed by summing the spatial receptive fields of the LGN cells that provide its input. Compare the result of summing appropriately placed LGN center-surround receptive fields (figure 2.27A) with the results of an appropriately adjusted Gabor filter model of the simple cell that uses the spatial kernel of

$$D_s(x,y)=\frac{1}{2\pi \sigma_x \sigma_y}\exp\left(-\frac{x^2}{2\sigma_x^2}-\frac{y^2}{2\sigma_y^2}\right)\cos(kx-\phi)$$

In [15]:
% TODO

## Problem 16
Construct the Hubel-Wiesel model of a complex cell, as depicted in figure 2.27B. Use Gabor functions given by

$$D_s(x,y)=\frac{1}{2\pi \sigma_x \sigma_y}\exp\left(-\frac{x^2}{2\sigma_x^2}-\frac{y^2}{2\sigma_y^2}\right)\cos(kx-\phi)$$

to model the simple cell responses, which should be rectified before being summed. Plot the spatial receptive field of the complex cell constructed by summing the different simple cell responses. Compare the responses of a complex cell constructed by linearly summing the outputs of simple cells (figure 2.27B) with different spatial phase preferences with the complex cell model obtained by squaring and summing two unrectified simple cell responses with spatial phases $90^\circ$ apart as in exercise 8.

In [16]:
% TODO